In [ ]:
import pandas as pd
import psycopg2 
import os
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 10000

run  `ssh -L 5433:localhost:5432 cardano-cli@node-1.mainnet.ctl-runtime.staging.mlabs.city -N` 
to establish ssh tunnel


or to forward socket:
`mkdir /tmp/sql-socket`
`ssh -L /tmp/sql-socket/.s.PGSQL.5432:/run/postgresql/.s.PGSQL.5432 -N cardano-cli@node-1.mainnet.ctl-runtime.staging.mlabs.city`
`psql -h /tmp/sql-socket -U cexplorer -d cdbsync`


In [ ]:
conn = psycopg2.connect(
    dbname='cdbsync',
    user='cexplorer',
    host='/tmp/sql-socket',
    port=5432
)

# conn.close()


In [ ]:
all_tables_and_columns = """
SELECT
    t.tablename AS table_name,
    a.attname AS column_name
FROM
    pg_tables t
JOIN
    pg_attribute a ON t.tablename = a.attrelid::regclass::text
WHERE
    t.tableowner = 'cexplorer'
    AND a.attnum > 0
    AND NOT a.attisdropped
ORDER BY
    t.tablename,
    a.attnum;
"""
all_tables = pd.read_sql(all_tables_and_columns,conn)

In [ ]:
all_tables

In [ ]:
max_time = """
SELECT max(time)
    
FROM
    block
;
"""
pd.read_sql(max_time,conn)

In [ ]:
blocks = """
SELECT
    *
FROM
    block
WHERE
  DATE(time)='2022-04-25'
LIMIT
    3;
"""
pd.read_sql(blocks,conn)

In [ ]:

blocks = """
SELECT
    ENCODE(tx.hash, 'hex') AS hash,
    block.block_no,
    block.time,
    block.tx_count,
    tx.script_size,
    tx.size
    
FROM block
LEFT JOIN tx ON block.id = tx.block_id    
WHERE
  DATE(block.time)='2022-04-25'

"""
df = pd.read_sql(blocks,conn)

In [ ]:
df.sort_values(by = 'size').head(2)


In [ ]:
df.sort_values(by = 'script_size').head(2)

In [ ]:
df.to_pickle('df.pkl')

In [ ]:

temp = """
SELECT
    ENCODE(tx.hash, 'hex') AS hash,
    tx.script_size,
    tx.size
    
    
FROM tx
WHERE script_size > 12000
LIMIT 10;
"""
pd.read_sql(temp,conn)

In [ ]:

temp = """
SELECT
  ENCODE(hash, 'hex') AS hash,  
  -- ENCODE(bytes, 'hex') AS bytes,  
  *
    
FROM script
WHERE type='plutusV2'
LIMIT 10;

"""
pd.read_sql(temp,conn)

In [ ]:

temp = """
SELECT
  distinct type
    
FROM script



"""
pd.read_sql(temp,conn)

In [ ]:

temp = """
SELECT
  *
FROM tx_in
WHERE tx_id=54531089




"""
pd.read_sql(temp,conn)